# Function calling OpenAI API


### How to define a function?


```python
function_definition = [{
    "type": "function",
    "function": {
        "name": "extract_job_info",
        "description": "Get the job information from the body of the input text,
        "parameters": {
            "type": "object",
            "properties": {
                "job": { "type": "string", "description": "Job Title" },
                "location": {"type": "string", "description": "Office location" }
            }
        }
    }
}]
```


# Function Calling example


In [ ]:
print("Hello")

In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv("../.env")

client = OpenAI()

MODEL_DEFAULTS = {
    "model": "gpt-5-mini",
}


# Test function
def search_job(query: str) -> str:
    return f"""
    Job listing for the query: {query}
    - Software Engineer
    - Data Engineer
    - DevOps Engineer
    - Cloud Engineer
    - Security Engineer
    - Site Reliability Engineer
    - Cloud Security Engineer
    """


# Define the tools
tools = [
    {
        "type": "function",
        "name": "search_job",
        "description": "Search for a job",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "job description"}
            },
            "required": ["query"],
        },
    }
]

MODEL_DEFAULTS["tools"] = tools

tool_mapping = {
    "search_job": search_job,
}

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can search for jobs",
    },
    {
        "role": "user",
        "content": "Developer in San Francisco",
    },
]

response = client.responses.create(
    input=messages,
    **MODEL_DEFAULTS,
)

# Filter the function calls
function_calls = [item for item in response.output if item.type == "function_call"]
new_message_items = []

for function_call in function_calls:
    target_tool = tool_mapping.get(function_call.name)
    if not target_tool:
        raise ValueError(f"Tool {function_call.name} not found")
    args = json.loads(function_call.arguments)
    print(f"Executing the tool: {function_call.name}")
    tool_output = target_tool(**args)
    new_message_items.append(
        {
            "type": "function_call_output",
            "call_id": function_call.call_id,
            "output": tool_output,
        }
    )

print("Calling the model again")
response = client.responses.create(
    model="gpt-5-mini",
    input=new_message_items,
    previous_response_id=response.id,
)
print(response.output_text)

Executing the tool: search_job
Calling the model again
I found these developer job types in San Francisco:
- Software Engineer
- Data Engineer
- DevOps Engineer
- Cloud Engineer
- Security Engineer
- Site Reliability Engineer
- Cloud Security Engineer

Which would you like me to search for specifically? Also tell me any of the following preferences so I can narrow results:
- Seniority (junior, mid, senior, staff/lead)
- Tech stack (e.g., JavaScript/React, Python, Go, Java, mobile)
- Remote / hybrid / on-site
- Full-time vs contract
- Target salary range or specific companies

I can then pull live openings with links, draft a tailored resume/cover letter, or set up job alerts. What do you want next?
